# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession, Row

sc = pyspark.SparkContext(appName='clickstream')
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 18:09:00,482 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from operator import add
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [3]:
!hadoop fs -put clickstream.csv clickstream.csv

put: `clickstream.csv': File exists


In [4]:
clickstream = se.read.option("delimiter", "\t").csv("clickstream.csv", header=True, inferSchema=True)
clickstream.registerTempTable("clickstream")
clickstream.show(1)

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|    562|       507|      page|      main|1695584127|
+-------+----------+----------+----------+----------+
only showing top 1 row



## Spark SQL 

### Process of getting the solution

First of all detect errors

In [5]:
error_query = """
SELECT user_id, session_id, MIN(timestamp) as error_timestamp
    FROM clickstream
    WHERE event_type LIKE '%error%'
    GROUP BY user_id, session_id
"""

se.sql(error_query).show(15)

+-------+----------+---------------+
|user_id|session_id|error_timestamp|
+-------+----------+---------------+
|   3705|       134|     1696280734|
|    957|       179|     1696199592|
|   1387|       218|     1696333010|
|   2025|       981|     1696475245|
|   2211|       900|     1696513086|
|   4868|       835|     1696566211|
|   3973|       515|     1696663564|
|   1150|       785|     1696675298|
|     75|       200|     1696694803|
|    352|        68|     1696703985|
|   3567|       733|     1696732810|
|    854|       241|     1696954446|
|   2165|       760|     1696364416|
|   3546|       518|     1696433407|
|    499|       687|     1696434083|
+-------+----------+---------------+
only showing top 15 rows



After that we need to filter it

In [6]:
filter_error_query = """
WITH ErrorTime AS (
    SELECT user_id, session_id, MIN(timestamp) as error_timestamp
    FROM clickstream
    WHERE event_type LIKE '%error%'
    GROUP BY user_id, session_id
)
SELECT 
    c.user_id, 
    c.session_id, 
    c.event_page, 
    c.timestamp,
    LAG(c.event_page, 1, 'start') OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp) AS lagged_event
FROM clickstream c
LEFT JOIN ErrorTime e 
    ON c.user_id = e.user_id 
    AND c.session_id = e.session_id
WHERE c.event_type = 'page' 
AND (c.timestamp < e.error_timestamp OR e.error_timestamp IS NULL);
"""

se.sql(filter_error_query).show(15)

+-------+----------+----------+----------+------------+
|user_id|session_id|event_page| timestamp|lagged_event|
+-------+----------+----------+----------+------------+
|      0|       874|      main|1696371064|       start|
|      0|       874|    rabota|1696374894|        main|
|      0|       874|    online|1696378229|      rabota|
|      0|       898|      main|1697594437|       start|
|      0|       898|      news|1697596341|        main|
|      0|       898|   tariffs|1697598240|        news|
|      0|       898|    rabota|1697600131|     tariffs|
|      0|       898|     bonus|1697605450|      rabota|
|      0|       898|   tariffs|1697611362|       bonus|
|      0|       898|     bonus|1697612492|     tariffs|
|      0|       898|  internet|1697614807|       bonus|
|      0|       898|      news|1697618742|    internet|
|      0|       898|   tariffs|1697619069|        news|
|      0|       898|    online|1697619756|     tariffs|
|      0|       898|   archive|1697624953|      

### The solution

In [7]:
routes_query = """
WITH ErrorTime AS (
    SELECT 
        user_id, 
        session_id, 
        MIN(timestamp) AS error_timestamp
    FROM clickstream
    WHERE event_type LIKE '%error%'
    GROUP BY user_id, session_id
),
FilteredActions AS (
    SELECT 
        c.user_id, 
        c.session_id, 
        c.event_page, 
        c.timestamp,
        LAG(c.event_page, 1, 'start') OVER (PARTITION BY c.user_id, c.session_id ORDER BY c.timestamp) AS lagged_event
    FROM clickstream c
    LEFT JOIN ErrorTime e 
        ON c.user_id = e.user_id 
        AND c.session_id = e.session_id
    WHERE c.event_type = 'page' 
    AND (c.timestamp < e.error_timestamp OR e.error_timestamp IS NULL)
),
ConcatenatedRoutes AS (
    SELECT 
        user_id, 
        session_id, 
        CONCAT_WS('-', COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp)) AS event_route, 
        ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp DESC) AS row_num
    FROM FilteredActions
    WHERE event_page != lagged_event OR lagged_event = 'start'
)
SELECT event_route, COUNT(*) AS route_count
FROM ConcatenatedRoutes
WHERE row_num = 1
GROUP BY event_route
ORDER BY route_count DESC
LIMIT 30;
"""

top_routes_sql = se.sql(routes_query)
top_routes_sql.show(30)

+--------------------+-----------+
|         event_route|route_count|
+--------------------+-----------+
|                main|       8184|
|        main-archive|       1113|
|         main-rabota|       1047|
|       main-internet|        897|
|          main-bonus|        870|
|           main-news|        769|
|        main-tariffs|        677|
|         main-online|        587|
|          main-vklad|        518|
| main-rabota-archive|        170|
| main-archive-rabota|        167|
|  main-bonus-archive|        143|
|   main-rabota-bonus|        139|
|   main-bonus-rabota|        135|
|    main-news-rabota|        135|
|main-archive-inte...|        132|
|    main-rabota-news|        130|
|main-internet-rabota|        129|
|   main-archive-news|        126|
|main-rabota-internet|        124|
|main-internet-arc...|        123|
|  main-archive-bonus|        117|
| main-internet-bonus|        115|
|main-tariffs-inte...|        114|
|   main-news-archive|        113|
|  main-news-interne

In [8]:
top_routes_sql.toPandas().to_csv("top_routes_sql.csv",sep = "\t", index = False)

In [9]:
import json
route_dict = {row['event_route']: row['route_count'] for row in top_routes_sql.limit(10).collect()}

# Write the result to a JSON file
with open('result.json', 'w') as json_file:
    json.dump(route_dict, json_file, indent=4)

## Spark RDD

### Transform to RDD

In [10]:
clickstream_rdd = clickstream.rdd

In [11]:
clickstream_rdd.take(10)

[Row(user_id=562, session_id=507, event_type='page', event_page='main', timestamp=1695584127),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584134),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584144),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584147),
 Row(user_id=562, session_id=507, event_type='wNaxLlerrorU', event_page='main', timestamp=1695584154),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584154),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584154),
 Row(user_id=562, session_id=507, event_type='event', event_page='main', timestamp=1695584160),
 Row(user_id=562, session_id=507, event_type='page', event_page='rabota', timestamp=1695584166),
 Row(user_id=562, session_id=507, event_type='event', event_page='rabota', timestamp=1695584174)]

### Creating mappers 

In [25]:
def map_events(row):
    # Create a key from user_id and session_id
    key = (row.user_id, row.session_id)
    
    # Prepare the value with timestamp and event information
    value = (row.timestamp, [row.event_type, row.event_page, 1 if 'error' in row.event_type else -1])
    
    return key, value

clickstream_grouped = (clickstream_rdd
    .map(map_events)
    .groupByKey() 
    .mapValues(list)
    .reduceByKey(lambda a, b: a + b)
    .mapValues(lambda records: sorted(records, key=lambda x: x[0]))
                      )


In [27]:
clickstream_grouped.take(1)

[((2209, 541),
  [(1695585442, ['page', 'main', -1]),
   (1695585480, ['event', 'main', -1]),
   (1695585553, ['page', 'bonus', -1]),
   (1695585571, ['event', 'bonus', -1]),
   (1695585593, ['page', 'online', -1]),
   (1695585610, ['event', 'online', -1]),
   (1695585733, ['event', 'online', -1]),
   (1695585783, ['event', 'online', -1]),
   (1695585792, ['page', 'internet', -1]),
   (1695585812, ['event', 'internet', -1]),
   (1695585822, ['event', 'internet', -1]),
   (1695585842, ['event', 'internet', -1]),
   (1695585878, ['page', 'news', -1]),
   (1695585918, ['event', 'news', -1]),
   (1695585934, ['event', 'news', -1]),
   (1695585951, ['event', 'news', -1]),
   (1695585994, ['page', 'main', -1]),
   (1695586015, ['event', 'main', -1])])]

In [28]:
def error_lag_mapper(x):
    is_error = False
    lagged_route = None

    for session in x[1]:  
        if is_error:
            session[1][2] = 1 
        elif session[1][2] > -1:
            is_error = True
            session[1][2] = 1

        if lagged_route is None:
            lagged_route = session[1][1] 
            session[1].append(None)
        else:
            session[1].append(lagged_route)
            lagged_route = session[1][1]

    return x

In [29]:
clickstream_grouped.map(error_lag_mapper).take(1)

[((2209, 541),
  [(1695585442, ['page', 'main', -1, None]),
   (1695585480, ['event', 'main', -1, 'main']),
   (1695585553, ['page', 'bonus', -1, 'main']),
   (1695585571, ['event', 'bonus', -1, 'bonus']),
   (1695585593, ['page', 'online', -1, 'bonus']),
   (1695585610, ['event', 'online', -1, 'online']),
   (1695585733, ['event', 'online', -1, 'online']),
   (1695585783, ['event', 'online', -1, 'online']),
   (1695585792, ['page', 'internet', -1, 'online']),
   (1695585812, ['event', 'internet', -1, 'internet']),
   (1695585822, ['event', 'internet', -1, 'internet']),
   (1695585842, ['event', 'internet', -1, 'internet']),
   (1695585878, ['page', 'news', -1, 'internet']),
   (1695585918, ['event', 'news', -1, 'news']),
   (1695585934, ['event', 'news', -1, 'news']),
   (1695585951, ['event', 'news', -1, 'news']),
   (1695585994, ['page', 'main', -1, 'news']),
   (1695586015, ['event', 'main', -1, 'main'])])]

In [30]:
def demapper(x):
    out = []
    for elem in x[1]:
        out.append((x[0], elem))
    return out


In [31]:
clickstream_grouped.map(error_lag_mapper).map(demapper).take(1)

[[((1974, 700), (1696091444, ['page', 'main', -1, None])),
  ((1974, 700), (1696091654, ['event', 'main', -1, 'main'])),
  ((1974, 700), (1696091785, ['page', 'online', -1, 'main'])),
  ((1974, 700), (1696092968, ['page', 'main', -1, 'online'])),
  ((1974, 700), (1696093073, ['event', 'main', -1, 'main'])),
  ((1974, 700), (1696093508, ['page', 'online', -1, 'main'])),
  ((1974, 700), (1696093548, ['page', 'rabota', -1, 'online'])),
  ((1974, 700), (1696093620, ['page', 'bonus', -1, 'rabota'])),
  ((1974, 700), (1696093655, ['event', 'bonus', -1, 'bonus'])),
  ((1974, 700), (1696093851, ['event', 'bonus', -1, 'bonus'])),
  ((1974, 700), (1696093918, ['page', 'archive', -1, 'bonus'])),
  ((1974, 700), (1696093967, ['event', 'archive', -1, 'archive'])),
  ((1974, 700), (1696094104, ['page', 'tariffs', -1, 'archive'])),
  ((1974, 700), (1696094159, ['page', 'bonus', -1, 'tariffs'])),
  ((1974, 700), (1696094341, ['event', 'bonus', -1, 'bonus'])),
  ((1974, 700), (1696094925, ['page', 'vkl

In [32]:
def get_route(x):
    routes = []
    for event in x[1]:
        routes.append(event[1][1])
    
    return [x[0], "-".join(routes)]


In [33]:
clickstream_grouped.map(error_lag_mapper).map(get_route).take(3)

[[(1974, 700),
  'main-main-online-main-main-online-rabota-bonus-bonus-bonus-archive-archive-tariffs-bonus-bonus-vklad-vklad-vklad-news-news-main-main-bonus-online-rabota-rabota-rabota-archive-archive-archive-archive-tariffs-tariffs-tariffs-tariffs-bonus-bonus-bonus-bonus-bonus-rabota-rabota-internet-internet-vklad-archive-archive-archive-archive-rabota'],
 [(2325, 433),
  'main-main-archive-archive-archive-archive-archive-archive-archive-archive-rabota-rabota-rabota-main-main-main-main-main-internet-internet-internet-vklad-vklad-vklad'],
 [(3496, 486),
  'main-bonus-bonus-internet-internet-internet-internet-archive-archive-archive-archive-archive-archive-archive-archive-bonus-bonus-tariffs-tariffs-rabota-rabota-rabota-rabota-rabota-news-archive']]

### Solution

In [39]:
clickstream_rdd_result = (clickstream_grouped
                            .map(error_lag_mapper)
                            .flatMap(demapper)
                            .filter(lambda x: x[1][1][2] == -1 and x[1][1][1] != x[1][1][3])
                            .groupByKey()
                            .map(lambda x: [x[0], list(x[1])])
                            .map(get_route)
                            .flatMap(lambda x: [(x[1], 1)])
                            .groupByKey()
                            .map(lambda x: (x[0], len(x[1])))
                            .sortBy(lambda x: -x[1])
                            .take(30)
                         )


In [40]:
clickstream_rdd_result

[('main', 8185),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-news-archive', 113),
 ('main-tariffs-internet', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-tariffs-archive', 103),
 ('main-internet-news', 103),
 ('main-rabota-main', 94)]

In [41]:
pd.DataFrame(clickstream_rdd_result, columns=['event_route', 'route_count']).to_csv("top_routes_rdd.csv",sep = "\t", index = False)

## Spark DF

In [ ]:
window_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

top_routes_df = (clickstream
                 ## join error timestamps
                   .join(
                       clickstream
                       .filter(clickstream.event_type.contains("error"))
                       .groupBy("user_id", "session_id")
                       .agg(F.min("timestamp").alias("error_timestamp")),
                       on=["user_id", "session_id"],
                       how="left"
                   )
                 ## filter errors
                   .filter(
                       (clickstream.event_type == 'page') &
                       ((clickstream.timestamp < F.col("error_timestamp")) | F.col("error_timestamp").isNull())
                   )
                 ## create lagged event column
                   .select(
                       clickstream.user_id, clickstream.session_id, 
                       clickstream.event_page, clickstream.timestamp,
                       F.lag(clickstream.event_page, 1, 'start').over(window_spec).alias('lagged_event')
                   )
                   .filter(
                       (F.col("event_page") != F.col("lagged_event")) | (F.col("lagged_event") == 'start')
                   )
                 ## create routes
                   .withColumn("event_route", 
                               F.concat_ws('-',
                                            F.collect_list("event_page").over(window_spec)))
                   .withColumn("row_num", 
                               F.row_number().over(window_spec.orderBy(F.desc("timestamp"))))
                 ## count routes
                   .filter(F.col("row_num") == 1)
                   .groupBy("event_route")
                   .agg(F.count("*").alias("route_count"))
                   .orderBy(F.desc("route_count"))
                   .limit(30))

In [ ]:
top_routes_df.show(30)

In [ ]:
top_routes_df.toPandas().to_csv("top_routes_df.csv",sep = "\t", index = False)